In [ ]:
!pip install transformers
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 85.4 MB/s 
     |████████████████████████████████| 7.6 MB 57.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 4.7 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#accumulated_grad = 2
epochs = 2
lr_init = 5e-5
batch_size = 4
adam_epsilon = 1e-8
warmup_steps = 3
block_size = 800
max_grad_norm = 1.0
model_save_interval = 2000
val_interval = 100

In [ ]:
import re
from tqdm import tqdm
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
#from transformers.tokenization_utils_base import BatchEncoding
import pandas as pd
import numpy as np
import os
from transformers import AdamW, get_linear_schedule_with_warmup
import tensorboardX as tb
from tensorboardX.summary import Summary
#from transformers import GPT2Tokenizer, GPT2LMHeadModel

## Data Cleaning(Skipped)(Ran on local machine)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
df_conversation = pd.read_csv('movie_scripts/movie_conversations.tsv', sep='\t', header=None)
df_movie = pd.read_csv('movie_scripts/movie_titles_metadata.tsv', sep='\t', header=None)

In [ ]:
import csv
df_list = []
with open("movie_scripts/movie_lines.tsv") as f:
   
  # Read data line by line
  for line in f:
     
    # split data by tab
    # store it in list
    l = line.split('\t')
    l1 = l[:4]
    l1.append(' '.join(l[4:]))
    # append list to ans
    df_list.append(l1)
df_line = pd.DataFrame(df_list, columns=['lineID','characterID','movieID','character name','utterance'])

In [ ]:
df_comedy_movie = df_movie.loc[df_movie[5].str.contains('comedy', na=False).values,:]

In [ ]:
df_comedy_conversation = df_conversation.loc[df_conversation[2].isin(df_comedy_movie[0].values),:].reset_index(drop=True)

In [ ]:
df_line['lineID'] = df_line['lineID'].str.replace(r'\"', '', regex=True)

In [ ]:
import re
final_comedy = []
for i in range(df_comedy_conversation.shape[0]):
    if (i+1)% 100 == 0:
        print('Step:',i+1)
    tmp_list = re.sub(r"\'", "", df_comedy_conversation[3][i].strip('[|]')).split()
    base_lineID = tmp_list[0]
    #print(base_lineID)
    base_utter = df_line[df_line.lineID.isin([base_lineID])]['utterance'].item()
    base_tokens = tokenizer.encode(base_utter) + [tokenizer.eos_token_id]
    for j in range(len(tmp_list)-1):
        response_lineID = tmp_list[j+1]
        response_utter = df_line[df_line.lineID.isin([response_lineID])]['utterance'].item()
        response_tokens = tokenizer.encode(response_utter) + [tokenizer.eos_token_id]
        base_tokens = base_tokens + response_tokens
        #final_comedy.append(base_tokens)  
    final_comedy.append(base_tokens)

In [ ]:
df_comedy_final = pd.DataFrame([str(tmp_list) for tmp_list in final_comedy], columns=['utter'])

In [ ]:
#df_comedy_final.to_csv('comedy_tokens.csv', index=False)
df_comedy_final.to_csv('comedy_tokens_simplified.csv', index=False)

### Filtering out Specific Sentences

In [ ]:
df_final = pd.read_csv('comedy_tokens_simplified_medium.csv')

In [ ]:
df_final = df_final['utter'].str.strip('[|]').str.split(', ')\
.apply(lambda x: [int(input_id) for input_id in x])

In [ ]:
df_utter = df_final.apply(lambda x: tokenizer.decode(x))

In [ ]:
import re
remove = ['No.', 'I don\'t know.', 'Yes.', 'I\'m sorry.']
df_utter_filtered = df_utter.apply(lambda x: re.sub('|'.join(['('+term+')' for term in remove]) ,'', x))
df_utter_filtered = df_utter.apply(lambda x: re.sub('<\|endoftext\|><\|endoftext\|>' ,'<\|endoftext\|>', x))
df_final_filtered = df_utter_filtered.apply(lambda x: tokenizer.encode(x))

In [ ]:
def has_continuous(x, token_id):
    for i in range(len(x)-1):
        if x[i]==token_id and x[i+1]==token_id:
            return True
    return False

In [ ]:
df_final_filtered.apply(lambda x: has_continuous(x, tokenizer.eos_token_id)).sum()

In [ ]:
df_final_filtered = df_final_filtered[df_final_filtered.apply(lambda x: len(x)>10)].reset_index(drop=True)

In [ ]:
df_final_filtered.to_csv('filtered_comedy_tokens_simplified_medium.csv', index=False)

## Reading csv

In [ ]:
test_length = 300

In [ ]:
df_final = pd.read_csv('/content/drive/MyDrive/data/Movie_Dialogpt/comedy_dialogues/comedy_tokens_simplified.csv')

In [ ]:
shuffled_df = df_final.sample(frac=1).reset_index(drop=True)

In [ ]:
shuffled_df['utter'] = shuffled_df['utter'].str.strip('[|]').str.split(', ')\
.apply(lambda x: [int(input_id) for input_id in x])

In [ ]:
df_train = shuffled_df.loc[:len(shuffled_df)-test_length-1]

In [ ]:
df_test = shuffled_df.loc[len(shuffled_df)-test_length:].reset_index(drop=True)

## Batch Data

In [ ]:
def batch_data(data):
    for epoch in range(epochs):
      for i in range(len(data)):
        X = data.loc[i]["utter"]
        yield X

## Model & Optimizer & Scheduler

In [ ]:
resume = False
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

if not resume:
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

    optimizer = AdamW(model.parameters(), lr=lr_init, eps=adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs*len(df_train)/batch_size)
else:
    model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/data/Movie_Dialogpt/small/models/training_args.bin')
     
    optimizer = AdamW(model.parameters(), lr=lr_init, eps=adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs*len(df_train)/batch_size)
    
    optimizer.load_state_dict(torch.load("/content/drive/MyDrive/data/Movie_Dialogpt/small/models/optimizer.pt"))
    scheduler.load_state_dict(torch.load("/content/drive/MyDrive/data/Movie_Dialogpt/small/models/scheduler.pt"))

if torch.cuda.is_available():
    device = torch.device('cuda:0') # GPU
else:
    device = torch.device('cpu') # CPU
model.to(device)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/351M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

## Training DialoGPT

In [ ]:
def evaluate(val_tokens, tb_writer, step):
    for i in range(val_tokens.shape[0]):
        tmp_tokens = val_tokens['utter'][i]
        stop = [i for i,e in enumerate(tmp_tokens) if e==tokenizer.eos_token_id][-2]
        base_tokens = torch.tensor([tmp_tokens[:stop+1]],device=device)
        true_text = tokenizer.decode(tmp_tokens, skip_special_tokens=True)
        predicted_tokens = model.generate(base_tokens, max_length=500)
        predicted_text = tokenizer.decode(predicted_tokens[0], skip_special_tokens=True)
        #print(predicted_tokens)
        print('### STEP: ', step+1)
        print('##### True_Text')
        print('\t'+true_text)
        print('##### Predicted Text')
        print('\t'+predicted_text)
        tb_writer.add_text(tag='val_true_text_'+str(i), text_string=true_text, global_step=step+1)
        tb_writer.add_text(tag='val_predicted_text'+str(i), text_string=predicted_text, global_step=step+1)

In [ ]:
### Tensorboard Summary & Folder Setup
folder_i = 0
while 1:
    if os.path.exists('/content/drive/MyDrive/data/Movie_Dialogpt/small/train_log/'+str(folder_i)):
        folder_i = folder_i + 1
    else:
        os.mkdir('/content/drive/MyDrive/data/Movie_Dialogpt/small/train_log/'+str(folder_i))
        break
tb_writer = tb.SummaryWriter('/content/drive/MyDrive/data/Movie_Dialogpt/small/train_log/'+str(folder_i))

### Training Process
tr_loss = 0.0
train_batches = batch_data(df_train)
for step, batch in tqdm(enumerate(train_batches)):
    #print('##step: ', step)
    inputs, labels = (batch, batch)
    if len(inputs) > 1024:
        print('#### Max Len Exceeded, Step: ', step+1)
        continue
    inputs = torch.tensor(inputs).to(device)
    labels = torch.tensor(inputs).to(device)
    model.train()
    outputs = model(inputs, labels=labels)
    loss = outputs[0]
    loss.backward()
    tb_writer.add_scalar(tag='train_loss', scalar_value=loss.item(), global_step=step+1)
    tr_loss += loss.item()
    
    if (step+1) % batch_size == 0:
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        model.zero_grad()
    if (step+1) % val_interval == 0:
        val_tokens = df_test.sample(2).reset_index(drop=True)
        evaluate(val_tokens, tb_writer, step)
    if (step+1) % model_save_interval == 0:
        model.save_pretrained('/content/drive/MyDrive/data/Movie_Dialogpt/small/models/training_args.bin')
        torch.save(optimizer.state_dict(), "/content/drive/MyDrive/data/Movie_Dialogpt/small/models/optimizer.pt")
        torch.save(scheduler.state_dict(), "/content/drive/MyDrive/data/Movie_Dialogpt/small/models/scheduler.pt")
tb_writer.close()

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
97it [00:06, 23.87it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architec

### STEP:  100
##### True_Text
	You want I nut him in the boat?
Yes? No. What?  Save the patter for the tourists.
You want me to let him in?
Of course dont be silly.

##### Predicted Text
	You want I nut him in the boat?
Yes? No. What?  Save the patter for the tourists.
You want me to let him in?

### STEP:  100
##### True_Text
	Look we can do other things.
I lived with you.  You were my partner just a few weeks ago and now you're worried I might kill you and you're entitled to worry.  Isn't that a terrible thing?  Isn't that sad?

##### Predicted Text
	Look we can do other things.



199it [00:10, 24.46it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
202it [00:11, 21.31it/s]

### STEP:  200
##### True_Text
	Hello Grandfather!
Hello.
He can talk then?

##### Predicted Text
	Hello Grandfather!
Hello.

### STEP:  200
##### True_Text
	You're barking up the wrong tree Hartman.
I'll give you three minutes to tell me where he is.

##### Predicted Text
	You're barking up the wrong tree Hartman.



281it [00:14, 25.50it/s]

...

48797it [40:17, 23.53it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


### STEP:  48800
##### True_Text
	There's something weird going on here Suzie.
No shit.
I'm serious. Get this. I had a careful look at that alternator. It's clearly been messed with... it's not wear and tear that caused that breakdown...
You're saying someone's trying to keep us on the farm...?
Not someone -- Mac.

##### Predicted Text
	There's something weird going on here Suzie.
No shit.
I'm serious. Get this. I had a careful look at that alternator. It's clearly been messed with... it's not wear and tear that caused that breakdown...
You're saying someone's trying to keep us on the farm...?
I'm not trying to keep you on the farm. I'm just saying that I'm not sure what's going on here.



48803it [40:18, 11.03it/s]

### STEP:  48800
##### True_Text
	I'm probably going to make out with her first before we you know get down...
You'll have to speak up son. I have this ringing in my ears. My doctor says it could be related to my heart palpitations. I've had two operations on my heart.
Really? I poop too much.
Oh maybe you're lactose intolerant.
Uh... No  I poop too much. Then I get tired.
Well if you find yourself getting tired take a couple of these.

##### Predicted Text
	I'm probably going to make out with her first before we you know get down...
You'll have to speak up son. I have this ringing in my ears. My doctor says it could be related to my heart palpitations. I've had two operations on my heart.
Really? I poop too much.
Oh maybe you're lactose intolerant.
Uh... No  I poop too much. Then I get tired.
I don't know what to do with that.



48856it [40:20, 20.18it/s]


## References

In [ ]:

# # Let's chat for 5 lines
# for step in range(1):
#     # encode the new user input, add the eos_token and return a tensor in Pytorch
#     new_user_input_ids = tokenizer.encode(user_inputs[step] + tokenizer.eos_token, return_tensors='pt')

#     # append the new user input tokens to the chat history
#     bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

#     # generated a response while limiting the total chat history to 1000 tokens, 
#     chat_history_ids = model.generate(bot_input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id, labels=bot_input_ids, temperature=1.0,\
# top_p? = 0.9)
#     print(chat_history_ids)
#     # pretty print last ouput tokens from bot
#     #print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

In [ ]:
# tokenizer.decode(model.generate(bot_input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id, labels=bot_input_ids)[0])

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'Does money buy happiness?<|endoftext|>Money buys happiness.<|endoftext|>'